<a href="https://colab.research.google.com/github/matheusvazdata/dados-lab/blob/main/APIs_authentication_and_interacting_with_the_world_wide_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Criando o arquivo .env e salvando a chave de API
with open(".env", "w") as f:
    f.write("OMDB_API_KEY=6b76310d")  # Substitua pela sua chave real

## Instalação e Configuração do `python-dotenv`

🔹 Isso permite carregar o .env dentro do Google Colab (onde está minha chave de autenticação da API).

In [2]:
!pip install python-dotenv

### Carregando a Chave da API do `.env`

In [3]:
# Importação das bibliotecas para recuperar a chave
import os
from dotenv import load_dotenv

# Carregamento das variáveis do .env
load_dotenv()

# Obtenção da chave da API do ambiente
chave_de_acesso = os.getenv("OMDB_API_KEY")

# Exibição para verificar se foi carregada corretamente
print("🔑 Chave da API carregada:", chave_de_acesso[:1] + "*******")

🔑 Chave da API carregada: 6*******


## Conectando-se a uma API com `requests`

In [4]:
# Importação das bibliotecas necessárias consumir e carregar os dados
import requests
import pandas as pd

#### OMDb (The Open Movie Database) API

🔗 Site para consultar a documentação, autenticar-se e saber mais: https://www.omdbapi.com/

In [5]:
# ID de um filme (Exemplo: "Guardians of the Galaxy Vol. 3")
id_filme = "tt6791350"

# Criação da URL com o ID e a chave da API
url = f"http://www.omdbapi.com/?i={id_filme}&apikey={chave_de_acesso}"

In [6]:
# Requisição feita
response = requests.get(url)

# Verificação do status da requisição
if response.status_code == 200:
    print("✅ Requisição bem-sucedida!")
else:
    print(f"❌ Erro na requisição! Status {response.status_code}.")

✅ Requisição bem-sucedida!


In [7]:
# Conversão do JSON para dicionário
dados = response.json()
dados  # Exibição dos detalhes do filme

{'Title': 'Guardians of the Galaxy Vol. 3',
 'Year': '2023',
 'Rated': 'PG-13',
 'Released': '05 May 2023',
 'Runtime': '150 min',
 'Genre': 'Action, Adventure, Comedy',
 'Director': 'James Gunn',
 'Writer': 'James Gunn, Jim Starlin, Stan Lee',
 'Actors': 'Chris Pratt, Chukwudi Iwuji, Bradley Cooper',
 'Plot': 'Still reeling from the loss of Gamora, Peter Quill rallies his team to defend the universe and one of their own - a mission that could mean the end of the Guardians if not successful.',
 'Language': 'English, Russian',
 'Country': 'United States, New Zealand, France, Canada',
 'Awards': 'Nominated for 1 Oscar. 11 wins & 88 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BOTJhOTMxMmItZmE0Ny00MDc3LWEzOGEtOGFkMzY4MWYyZDQ0XkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.9/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '82%'},
  {'Source': 'Metacritic', 'Value': '64/100'}],
 'Metascore': '64',
 'imdbRating': '7.9',
 'i

In [8]:
# Criação de um DataFrame com os dados
df = pd.DataFrame([dados])

# Exibição das informações relevantes
df[['Title', 'Year', 'Runtime', 'Genre', 'Language', 'Metascore', 'imdbRating']]

,Title,Year,Runtime,Genre,Language,Metascore,imdbRating
0,Guardians of the Galaxy Vol. 3,2023,150 min,"Action, Adventure, Comedy","English, Russian",64,7.9


### Buscar 100 filmes quaisquer

In [9]:
# Listagrm de palavras comuns para buscas
termos_busca = ['Horror', 'Kill', 'War', 'Night', 'Bad', 'Dark', 'Nightmare']
max_filmes = 100  # Número máximo de filmes desejados
filmes_completos = set()  # Uso de conjunto para evitar duplicatas
pagina = 1

In [10]:
# Criação de um loop para percorrer todos os termos de busca
for termo in termos_busca:
    pagina = 1
    while len(filmes_completos) < max_filmes:
        url = f"http://www.omdbapi.com/?s={termo}&page={pagina}&apikey={chave_de_acesso}"
        response = requests.get(url)
        dados = response.json()

        # Verificação se há resultados e adição ao conjunto
        if 'Search' in dados:
            for filme in dados['Search']:
                filmes_completos.add((filme['imdbID'], filme['Title'], filme['Year']))
                if len(filmes_completos) >= max_filmes:
                    break
            pagina += 1
        else:
            print(f"Erro: {dados.get('Error', 'Desconhecido')}")
            break  # Encerramento do loop caso não tenha mais resultados

#### Criação do DataFrame

In [11]:
# Criação do DataFrame inicial
df_filmes = pd.DataFrame(list(filmes_completos), columns=['imdbID', 'Title', 'Year'])
df_filmes.head()

,imdbID,Title,Year
0,tt1282045,Flu Bird Horror,2008
1,tt6873926,Two Sentence Horror Stories,2017–2022
2,tt15132736,Horror Noire,2021
3,tt0101699,Delicacies of Molten Horror Synapse,1991
4,tt11708140,Asylum: Twisted Horror and Fantasy Tales,2020


### Exploração das notas dos filmes (que estão associadas ao `imdbID`)

In [12]:
# Uso de time para evitar muitas requisições seguidas e numpy para preencher com nulos os filmes sem notas
import time
import numpy as np

In [13]:
# Busca das Notas IMDb
notas_imdb = []

for imdb_id in df_filmes['imdbID']:
    url_detalhes = f"http://www.omdbapi.com/?i={imdb_id}&apikey={chave_de_acesso}"
    response = requests.get(url_detalhes)
    detalhes = response.json()

    notas_imdb.append(detalhes.get("imdbRating", np.nan))  # Adição da nota ou "NaN" se não disponível
    time.sleep(0.5)  # 🔴 Evita sobrecarga de requisições (limite da API)

In [14]:
# Adição das notas ao DataFrame original
df_filmes['IMDb Rating'] = notas_imdb

# Exibição dos resultados
display(df_filmes[['Title', 'Year', 'imdbID', 'IMDb Rating']])

,Title,Year,imdbID,IMDb Rating
0,Flu Bird Horror,2008,tt1282045,3.0
1,Two Sentence Horror Stories,2017–2022,tt6873926,5.6
2,Horror Noire,2021,tt15132736,5.1
3,Delicacies of Molten Horror Synapse,1991,tt0101699,5.9
4,Asylum: Twisted Horror and Fantasy Tales,2020,tt11708140,4.8
...,...,...,...,...
95,The Horror of Party Beach,1964,tt0058208,3.3
96,The Zombie Horror Picture Show,2014,tt4036514,6.2
97,American Horror Stories,2021–,tt12306692,6.3
98,Rocky Horror Show Live,2015,tt5061126,8.1
